In [24]:
import pandas as pd
import os
from neo4j import GraphDatabase
import ..labdataranger as ldr

db_config = ldr.query.get_db_config()
uri = f"{db_config['uri']}:{db_config['port']}"
username = db_config['username']
password = db_config['password']
database = "instruments"

driver = GraphDatabase.driver(
    uri, 
    auth=(username, password)
)

def fetch_folders_with_substring(driver, fp_substring, database=None):
    query = """
        MATCH (f:Folder)
        WHERE f.name CONTAINS $filePathSubstring
        RETURN f.name, f.filepath
    """
    
    with driver.session(database=database) as session:
        result = session.run(query, filePathSubstring=fp_substring)
        # Convert results to DataFrame
        data = [{"name": record["name"], "filepath": record["filepath"]} 
                for record in result]
        df = pd.DataFrame(data)
        return df


In [19]:
query = """
    MATCH (f:Folder)
    WHERE f.name CONTAINS $filePathSubstring
    RETURN f.name, f.filepath
"""
fp_substring='_Rec'
df = None
data = None
result = None
with driver.session(database=database) as session:
    result = session.run(query, filePathSubstring=fp_substring)
    # Convert results to DataFrame
    data = [{"name": record["f.name"], "filepath": record["f.filepath"]} 
            for record in result]

    df = pd.DataFrame(data)

In [20]:
df

,name,filepath
0,2R_Rec,/mnt/data/archive/LaraM_Full_Dataset/U52/12Apr...
1,NP_Rec,/mnt/data/archive/LaraM_Full_Dataset/U52/05Apr...
2,1L_Rec,/mnt/data/archive/LaraM_Full_Dataset/U52/05Apr...
3,1R_Rec,/mnt/data/archive/LaraM_Full_Dataset/U52/05Apr...
4,2L_Rec,/mnt/data/archive/LaraM_Full_Dataset/U52/05Apr...
...,...,...
2775,0_Rec,/mnt/data/archive/LaraM_Full_Dataset/09.03.21_...
2776,2R_Rec,/mnt/data/archive/LaraM_Full_Dataset/09.03.21_...
2777,1L_Rec,/mnt/data/archive/LaraM_Full_Dataset/09.03.21_...
2778,1R_Rec,/mnt/data/archive/LaraM_Full_Dataset/09.03.21_...


In [21]:
df['filepath'] = df['filepath'].apply(lambda x: '/'.join(x.split('/')[4:]))

In [23]:
df.to_csv('recon_filepaths.csv')